# Literal Eval Conversions

In [2]:
# !pip install datasets
# !pip install transformers

In [35]:
import os
import json
import transformers
import pandas as pd
from ast import literal_eval
from datasets import load_dataset
from datasets import Dataset, ClassLabel, Sequence
from collections import OrderedDict
from sklearn.model_selection import train_test_split

In [4]:
system="COLAB"

if system=="COLAB":
    from google.colab import drive
    drive.mount("/content/gdrive")
    #DATA_DIR = os.path.join("/content/gdrive/My Drive", "NER/Data")
    DATA_DIR = os.path.join("/content/gdrive/Shareddrives/", "GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data")

Mounted at /content/gdrive


## 1. New Label Map

In [5]:
old_label_map = {'O': 0,
                'CONTACT': 1,
                'DATE': 2,
                'EVENT': 3,
                'FINANCE': 4,
                'FORM': 5,
                'LOCATION': 6,
                'MISC': 7,
                'MONEY': 8,
                'ORGANIZATION': 9,
                'PERSON': 10,
                'SCHEME': 11,
                'STATE': 12}

In [6]:
new_label_map_mapping = {'CONTACT': 'I-CONTACT',
                         'DATE': 'I-DATE',
                         'EVENT': 'I-EVENT',
                         'FINANCE': 'I-FINANCE',
                         'FORM': 'I-FORM',
                         'LOCATION': 'I-LOC',
                         'MISC': 'I-MISC',
                         'MONEY': 'I-MONEY',
                         'O': 'O',
                         'ORGANIZATION': 'I-ORG',
                         'PERSON': 'I-PER',
                         'SCHEME': 'I-SCHEME',
                         'STATE': 'I-STATE'}

## 2. Convert Label Convention

Read in dataset from CSV...

In [7]:
save_df_name = 'govuk-labelled-data-ner.csv'
save_df_path = f'{DATA_DIR}/{save_df_name}'
save_df_path

'/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/govuk-labelled-data-ner.csv'

In [8]:
all_data = pd.read_csv(save_df_path)

Evaluate literals...

In [9]:
all_data['text_token'] = all_data['text_token'].apply(lambda x: literal_eval(x))
all_data['labels'] = all_data['labels'].apply(lambda x: literal_eval(x))
all_data['label_list'] = all_data['label_list'].apply(lambda x: literal_eval(x))

In [10]:
all_data.head()

,text,text_token,labels,updated,original_labels,base_path,sampled,label_list,original_file,new_label_list_id
0,"However , to be clear , all of the other check...","[However, ,, to, be, clear, ,, all, of, the, o...","[[41, 47, EVENT], [81, 90, PERSON]]",NaN,NaN,/government/speeches/business-secretarys-state...,True,"[O, O, O, O, O, O, O, O, O, O, EVENT, O, O, O,...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ..."
1,Organisations that met certain criteria during...,"[Organisations, that, met, certain, criteria, ...","[[65, 69, DATE], [77, 85, FORM]]",NaN,NaN,/guidance/crc-energy-efficiency-scheme-qualifi...,True,"[O, O, O, O, O, O, O, O, DATE, O, O, FORM, O, ...",line_by_line_NER_data_sampled_09062020_more_en...,"[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 5, 0, 0, 0, ..."
2,Use this leaflet and form to find out about pa...,"[Use, this, leaflet, and, form, to, find, out,...","[[21, 25, FORM], [44, 50, FINANCE], [51, 83, F...",NaN,NaN,/government/publications/social-security-abroa...,True,"[O, O, O, O, FORM, O, O, O, O, FINANCE, FINANC...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 4, 4, 4, 0, 6, ..."
3,Legal visits must be booked by email : legalvi...,"[Legal, visits, must, be, booked, by, email, :...","[[31, 36, CONTACT], [82, 94, PERSON], [103, 10...",NaN,NaN,/guidance/liverpool-prison,True,"[O, O, O, O, O, O, CONTACT, O, O, O, O, O, PER...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 10, 10, 0..."
4,RPC checklist for Small and Micro Business Ass...,"[RPC, checklist, for, Small, and, Micro, Busin...","[[0, 3, ORGANIZATION], [34, 42, ORGANIZATION],...",NaN,NaN,/government/publications/small-and-micro-busin...,True,"[ORGANIZATION, O, O, O, O, O, ORGANIZATION, O,...",line_by_line_NER_data_sampled_12062020_more_en...,"[9, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 2, 2, 0]"


Casting the replacement tags. The tags will need replacing in the following columns...

In [11]:
df_IO = all_data

In [12]:
def list_rep(lists, lookup):
  new_list = []
  for l in lists:
    new_l = []
    for i in l:
      if isinstance(i, str):
        i = lookup[i]
      else:
        i = i
      new_l.append(i)
    new_list.append(new_l)
  return new_list

In [13]:
df_IO['labels'] = df_IO['labels'].apply(lambda x: list_rep(x, lookup=new_label_map_mapping))
df_IO['label_list'] = df_IO['label_list'].apply(lambda x: list(map(new_label_map_mapping.get, x)))

In [21]:
df_IO.head(10)

,text,text_token,labels,updated,original_labels,base_path,sampled,label_list,original_file,new_label_list_id
0,"However , to be clear , all of the other check...","[However, ,, to, be, clear, ,, all, of, the, o...","[[41, 47, I-EVENT], [81, 90, I-PER]]",NaN,NaN,/government/speeches/business-secretarys-state...,True,"[O, O, O, O, O, O, O, O, O, O, I-EVENT, O, O, ...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ..."
1,Organisations that met certain criteria during...,"[Organisations, that, met, certain, criteria, ...","[[65, 69, I-DATE], [77, 85, I-FORM]]",NaN,NaN,/guidance/crc-energy-efficiency-scheme-qualifi...,True,"[O, O, O, O, O, O, O, O, I-DATE, O, O, I-FORM,...",line_by_line_NER_data_sampled_09062020_more_en...,"[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 5, 0, 0, 0, ..."
2,Use this leaflet and form to find out about pa...,"[Use, this, leaflet, and, form, to, find, out,...","[[21, 25, I-FORM], [44, 50, I-FINANCE], [51, 8...",NaN,NaN,/government/publications/social-security-abroa...,True,"[O, O, O, O, I-FORM, O, O, O, O, I-FINANCE, I-...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 4, 4, 4, 0, 6, ..."
3,Legal visits must be booked by email : legalvi...,"[Legal, visits, must, be, booked, by, email, :...","[[31, 36, I-CONTACT], [82, 94, I-PER], [103, 1...",NaN,NaN,/guidance/liverpool-prison,True,"[O, O, O, O, O, O, I-CONTACT, O, O, O, O, O, I...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 10, 10, 0..."
4,RPC checklist for Small and Micro Business Ass...,"[RPC, checklist, for, Small, and, Micro, Busin...","[[0, 3, I-ORG], [34, 42, I-ORG], [66, 77, I-DA...",NaN,NaN,/government/publications/small-and-micro-busin...,True,"[I-ORG, O, O, O, O, O, I-ORG, O, O, O, O, O, I...",line_by_line_NER_data_sampled_12062020_more_en...,"[9, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 2, 2, 0]"
5,Applications are open from 4 July 2019 until 1...,"[Applications, are, open, from, 4, July, 2019,...","[[0, 12, I-FORM], [27, 38, I-DATE], [48, 51, I...",NaN,NaN,/government/publications/rescue-boat-grant-fun...,True,"[I-FORM, O, O, O, I-DATE, I-DATE, I-DATE, O, I...",line_by_line_NER_data_sampled_12062020_more_en...,"[5, 0, 0, 0, 2, 2, 2, 0, 8, 0, 2, 2, 2, 2, 0]"
6,He converted some of these weapons to fire liv...,"[He, converted, some, of, these, weapons, to, ...","[[92, 104, I-DATE]]",NaN,NaN,/government/news/sentence-increased-for-walsal...,True,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,Fortified foods : guidance to compliance with ...,"[Fortified, foods, :, guidance, to, compliance...","[[18, 26, I-FORM], [46, 54, I-ORG]]",NaN,NaN,/government/publications/fortified-foods-guida...,True,"[O, O, O, I-FORM, O, O, O, I-ORG, O, O, O, O, ...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 5, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0]"
8,This protocol sets out nationally agreed metho...,"[This, protocol, sets, out, nationally, agreed...","[[64, 71, I-PER], [97, 109, I-EVENT]]",NaN,NaN,/government/publications/diagnostic-radiology-...,True,"[O, O, O, O, O, O, O, O, O, I-PER, O, O, O, O,...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 3, 0]"
9,New Style JSA is a fortnightly payment that ca...,"[New, Style, JSA, is, a, fortnightly, payment,...","[[10, 13, I-FINANCE], [31, 38, I-FINANCE], [73...",NaN,NaN,/guidance/new-style-jobseekers-allowance,True,"[O, O, I-FINANCE, O, O, O, I-FINANCE, O, O, O,...",line_by_line_NER_data_sampled_09062020_more_en...,"[0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [22]:
samp_df_IO = df_IO.sample(10000)

Save main to file...

In [23]:
save_path = '/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/IO-annotation_scheme'
save_name = os.path.join(save_path, 'i-o_labelled_data.csv')
df_IO.to_csv(save_name, index=None)

Save sampled to file...

In [24]:
save_path = '/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/IO-annotation_scheme'
save_name = os.path.join(save_path, 'i-o_labelled_data_samp.csv')
samp_df_IO.to_csv(save_name, index=None)

Save mappings to json file 

In [25]:
label_map = {'O': 0,
            'I-CONTACT': 1,
            'I-DATE': 2,
            'I-EVENT': 3,
            'I-FINANCE': 4,
            'I-FORM': 5,
            'I-LOC': 6,
            'I-MISC': 7,
            'I-MONEY': 8,
            'I-ORG': 9,
            'I-PER': 10,
            'I-SCHEME': 11,
            'I-STATE': 12}

In [26]:
save_path = '/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/IO-annotation_scheme'
save_name = os.path.join(save_path, 'i-o_label_map.json')

In [27]:
with open(save_name, 'w') as fp:
    json.dump(label_map, fp)

## 3. Huggingface Preparation

### 3.1 Main Data

In [53]:
file_path = '/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/IO-annotation_scheme'
file_name = os.path.join(save_path, 'i-o_labelled_data.csv')
df = pd.read_csv(file_name)

In [54]:
df_trim = df[['text_token', 'new_label_list_id']]
df_trim.head()

,text_token,new_label_list_id
0,"['However', ',', 'to', 'be', 'clear', ',', 'al...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ..."
1,"['Organisations', 'that', 'met', 'certain', 'c...","[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 5, 0, 0, 0, ..."
2,"['Use', 'this', 'leaflet', 'and', 'form', 'to'...","[0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 4, 4, 4, 0, 6, ..."
3,"['Legal', 'visits', 'must', 'be', 'booked', 'b...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 10, 10, 0..."
4,"['RPC', 'checklist', 'for', 'Small', 'and', 'M...","[9, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 2, 2, 0]"


In [52]:
hf_dataset = Dataset.from_pandas(df)

In [45]:
labels = [i for i in label_map.keys()]
labels

['O',
 'I-CONTACT',
 'I-DATE',
 'I-EVENT',
 'I-FINANCE',
 'I-FORM',
 'I-LOC',
 'I-MISC',
 'I-MONEY',
 'I-ORG',
 'I-PER',
 'I-SCHEME',
 'I-STATE']

In [46]:
for j in ['text_token', 'new_label_list_id']:
  print("{}: {}".format(j, hf_dataset.features[f"{j}"]))
print()

hf_dataset.features['new_label_list_id'] = Sequence(ClassLabel(13, labels), -1 , id=None)

text_token: Value(dtype='string', id=None)
new_label_list_id: Sequence(feature=ClassLabel(num_classes=13, names=['O', 'I-CONTACT', 'I-DATE', 'I-EVENT', 'I-FINANCE', 'I-FORM', 'I-LOC', 'I-MISC', 'I-MONEY', 'I-ORG', 'I-PER', 'I-SCHEME', 'I-STATE'], names_file=None, id=None), length=-1, id=None)



In [47]:
hf_dataset = hf_dataset.train_test_split(train_size=0.85, seed=42)

ValueError: ignored

In [31]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'text_token', 'labels', 'updated', 'original_labels', 'base_path', 'sampled', 'label_list', 'original_file', 'new_label_list_id'],
        num_rows: 295133
    })
    test: Dataset({
        features: ['text', 'text_token', 'labels', 'updated', 'original_labels', 'base_path', 'sampled', 'label_list', 'original_file', 'new_label_list_id'],
        num_rows: 52083
    })
})

In [ ]:
dataset_name = "hf_govuk_data_i-o"

In [ ]:
dataset_name_path = f'{file_path}/{dataset_name}'
dataset_name_path

'/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/IO-annotation_scheme/hf_govuk_data_i-o'

In [ ]:
# save train_dataset to gdrive
hf_input_path = f'{dataset_name_path}'
hf_dataset.save_to_disk(hf_input_path)

Flattening the indices:   0%|          | 0/296 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/53 [00:00<?, ?ba/s]

### 3.2 Sampled Data

In [ ]:
file_path = '/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/IO-annotation_scheme'
file_name = os.path.join(save_path, 'i-o_labelled_data_samp.csv')
df = pd.read_csv(file_name)

In [ ]:
hf_dataset = Dataset.from_pandas(df)

In [ ]:
hf_dataset = hf_dataset.train_test_split(train_size=0.85, seed=42)

In [ ]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'text_token', 'labels', 'updated', 'original_labels', 'base_path', 'sampled', 'label_list', 'original_file', 'new_label_list_id'],
        num_rows: 8500
    })
    test: Dataset({
        features: ['text', 'text_token', 'labels', 'updated', 'original_labels', 'base_path', 'sampled', 'label_list', 'original_file', 'new_label_list_id'],
        num_rows: 1500
    })
})

In [ ]:
for j in ['text_token', 'new_label_list_id']:
  print("{}: {}".format(j, hf_dataset.features[f"{j}"]))
print()

labels = [i for i in label_map.keys()]

hf_dataset.features['new_label_list_id'] = Sequence(ClassLabel(13, labels), -1 , id=None)

In [ ]:
dataset_name = "samp_hf_govuk_data_i-o"

In [ ]:
dataset_name_path = f'{file_path}/{dataset_name}'
dataset_name_path

'/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/IO-annotation_scheme/samp_hf_govuk_data_i-o'

In [ ]:
# save train_dataset to gdrive
hf_input_path = f'{dataset_name_path}'
hf_dataset.save_to_disk(hf_input_path)

Flattening the indices:   0%|          | 0/9 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/2 [00:00<?, ?ba/s]